### Subscriptions

#### Appointments
```
GET

​/v1​/{practiceid}​/appointments​/changed​/subscription

Get list of appointment slot change subscription(s)

POST

​/v1​/{practiceid}​/appointments​/changed​/subscription

Subscribe to all/specific change events for appointment slots
```

#### Allergies
```
POST

​/v1​/{practiceid}​/chart​/healthhistory​/allergies​/changed​/subscription

Subscribe to all/specific change events for allergies


GET

​/v1​/{practiceid}​/chart​/healthhistory​/allergies​/changed​/subscription

Get list of allergy change subscription(s)


GET

​/v1​/{practiceid}​/chart​/healthhistory​/allergies​/changed​/subscription​/events

Get list of allergy change events to which you are subscribed
```


#### Medication
```
POST

​/v1​/{practiceid}​/chart​/healthhistory​/medication​/changed​/subscription

Subscribe to all/specific change events for medication list


GET

​/v1​/{practiceid}​/chart​/healthhistory​/medication​/changed​/subscription

Get list of medication list change subscription(s)


GET

​/v1​/{practiceid}​/chart​/healthhistory​/medication​/changed​/subscription​/events

Get list of medication list change events to which you can subscribe
```

#### Problem
```
POST

​/v1​/{practiceid}​/chart​/healthhistory​/problems​/changed​/subscription

Subscribe to all/specific change events for problems


GET

​/v1​/{practiceid}​/chart​/healthhistory​/problems​/changed​/subscription

Get list of problems change subscription(s)


DELETE

​/v1​/{practiceid}​/chart​/healthhistory​/problems​/changed​/subscription

Unsubscribe to all/specific change events for problems


GET
​
/v1​/{practiceid}​/chart​/healthhistory​/problems​/changed​/subscription​/events

Get list of problems change events to which you can subscribe
```

#### Vaccines
```
GET

​/v1​/{practiceid}​/chart​/healthhistory​/vaccine​/changed

Get list of changes in vaccines based on subscription


POST

​/v1​/{practiceid}​/chart​/healthhistory​/vaccine​/changed​/subscription

Subscribe to all/specific change events for vaccines


GET

​/v1​/{practiceid}​/chart​/healthhistory​/vaccine​/changed​/subscription

Get list of vaccine change subscription(s)


DELETE

​/v1​/{practiceid}​/chart​/healthhistory​/vaccine​/changed​/subscription

Unsubscribe to all/specific change events for vaccines


GET

​/v1​/{practiceid}​/chart​/healthhistory​/vaccine​/changed​/subscription​/events

Get list of vaccine change events to which you can subscribe
```

#### Additional available resources
##### Documents and Forms
Documents / Imaging Results / Lab Results / Orders / Prescriptions

##### Hospital
Charge Codes / OR Case / Stays / Surgery / Visits 

##### Insurance
CCM Enrollments / Claims

##### Provider



## Subscription Endpoint Responses
[Appointments](./sandbox_responses/subscription_appointments.json)


[Patients](./sandbox_responses/subscription_patients.json)

## TODOs
* Link to CS FHIR client
* Post under cs_athena in EHR-Modules


In [ ]:
import logging
import pprint
import os
from fhirclient import client

In [ ]:
settings = {
    'app_id': os.getenv('APP_ID'),
    'app_secret': os.getenv('APP_SECRET'),
    'api_base': 'https://api.platform.athenahealth.com/fhir/r4',
    'scope': os.getenv('APP_SCOPE')
}

In [ ]:
smart = client.FHIRClient(settings=settings)
smart.authorize_url

In [ ]:
#dstu = client.FHIRClient(settings=dstu_settings)
#dstu.authorize_url

In [ ]:
smart.authorize()

## Patient

In [ ]:
import fhirclient.models.patient as p
patient = p.Patient.read('a-18470.E-73', smart.server)
#print(patient.id)
#print(patient.name[0].given[0])
pprint.pprint(patient.as_json())

In [ ]:
for a in patient.address:
    print()
    print(a.line[0])

## Appointment - FHIR DSTU2

[Day's appointments](./sandbox_responses/appointments.json)

In [ ]:
# ERROR - DSTU2
'''
import fhirclient.models.appointment as appointment
appointments = appointment.Appointment.read(274032, smart.server)
#appointments = appointment_search.perform_resources(smart.server)

for a in appointments:
    pprint.pprint(a.as_json())
'''

## Allergy

In [ ]:
import fhirclient.models.allergyintolerance as allergy
search = allergy.AllergyIntolerance.where(struct={'patient': patient.id})
ai = search.perform_resources(smart.server)

for patient_allergy in ai:
    pprint.pprint(patient_allergy.as_json())

## Immunization

* Can't find the actual medicine administered.  May need to look at encounters

In [ ]:
import fhirclient.models.immunization as immunization
immunization_search = immunization.Immunization.where(struct={'patient': patient.id})
immunizations = immunization_search.perform_resources(smart.server)

for i in immunizations:
    pprint.pprint(i.as_json())

## Encounter - DSTU2
/v1/{practiceid}/{departmentid}/fhir/dstu2/Encounter

* Doesn't necessarilly translate to an appointment
* Doesn't have an appointment ID

[Encounters](./sandbox_responses/encounters.json)

## Insurance - V1 endpoints

/v1/{practiceid}/patients/{patientid}/insurances

[Insurance](./sandbox_responses/insurance.json)

## MedicationRequest

[Sandbox MedicationRequest Reponse](./sandbox_responses/medicationRequest.json)

Responses vary.  Examples:

`amoxicillin 875 mg-potassium clavulanate 125 mg tablet`


In [ ]:
# ERROR
import fhirclient.models.medicationrequest as medication
medication_search = medication.MedicationRequest.where(struct={'patientid': patient.id, 'ah-practice': 'Organization/a-1.Practice-18470', 'intent': 'order', })
medications = medication_search.perform_resources(smart.server)

for m in medications:
    pprint.pprint(m.as_json())

## Problem - FHIR Conditions

In [ ]:
import fhirclient.models.condition as condition
condition_search = condition.Condition.where(struct={'patient': patient.id})
conditions = condition_search.perform_resources(smart.server)

for c in conditions:
    pprint.pprint(c.as_json())

## Vitals - FHIR Observation
* Data is inconsistent
* Getting multiples of the same parameter(for instance HR)
* Did not find bp in any of the records

In [ ]:
import fhirclient.models.observation as observation
observation_search = observation.Observation.where(struct={'patient': patient.id})
observations = observation_search.perform_resources(smart.server)

for o in observations:
    pprint.pprint(o.as_json())
    o.category

## Vitals - /v1/
/v1/{practiceid}/chart/{patientid}/vitals

[v1/18470/chart/73/vitals?departmentid=1](./sandbox_responses/vitals.json)

## Organization

In [ ]:
import fhirclient.models.organization as o
org = o.Organization.read('a-1.Practice-18470', smart.server)
org.as_json()